### Player Outcomes

- The goal of this notebook is to take in the current contest data and produce the results for the following:
    - `proj-fpts` : median projection fantasy points (FP)
    - `ceiling-fpts`: 75% outcome of FP
    - `floor-fpts`: 25% outcome of FP
    
    - `boom-prob`: likliehood of player achieving *at least* 5x+10 FP
    - `bust-prob`: likliehood of player achieving *at most* 5x FP
    - `neutral-prob`: likliehood of player achieving between (5x, 5x+10) FP

- NOTES: these projections will not go to into depth, therefore results will not include outcomes for those specific lineups but rather just players for whole of season.
- This may cause issues for players who are missing high FP scoring players for current contest, players who have past outcomes distorted by long durations of team missing high FP scoring players, and for players who have been traded.

In [1]:
import numpy as np
import pandas as pd

In [2]:
def pandas_settings() -> None:
        for option in ('display.max_rows', 'display.max_columns', 'display.width', 'display.memory_usage'):
            pd.set_option(option, 250 if 'memory_usage' not in option else False)
        # message('pandas')
        return None
    
pandas_settings()

In [3]:

# Helper function to pass into .agg for a groupby
# Calculates percentile outcome of given np array ()
# Example: percentile(50) = np.median
def percentile(n: int) -> float:
    def percentile_(arr):
        return np.percentile(arr, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [4]:
class Outcomes:
    def __init__(self, **kwargs):
        
        # Need to convert fantasy points if draftkings
        mode: str = kwargs.get('mode', 'fanduel')
        
        stats_: list[str,...] = [
            'name',
            'date',
            'team',
            'opp',
            'fpts',
            'mp',
            'fppm',
            'usg',
            'pts',
            'ast',
            'trb',
            'stl',
            'blk',
            'tov',
            'starter',
            'ast_perc',
            '3p'
        ]
        
        stats = sum([
            stats_,
            [stat for stat in kwargs.get('stats', list()) if stat not in stats_]
        ], list())
        
        self.szn: pd.DataFrame = (pd
                                  .read_csv('../data/season-data-clean.csv')
                                  .pipe(lambda df_: df_.loc[(df_['mp'] >= 8.0)])
                                  [stats]
                                 )
        
        fd_columns: dict[str, str] = {
            'Nickname': 'name',
            'Position': 'pos',
            'Team': 'team',
            'Salary': 'salary',
        }
        
        dk_columns: dict[str, str] = {
            'Name': 'name',
            'Roster Position': 'pos',
            'TeamAbbrev': 'team',
            'Salary': 'salary'
        }
        
        columns: dict[str, str] = fd_columns if mode=='fanduel' else dk_columns
        
        # Only fanduel gives injuries
        injured_players: tuple[str,...] = tuple(pd
                                                .read_csv('../data/current-fanduel.csv', usecols=['Nickname', 'Injury Indicator'])
                                                .rename({'Nickname': 'name', 'Injury Indicator': 'injury'}, axis=1)
                                                .pipe(lambda df_: df_.loc[df_['injury']=='O'])
                                                ['name']
                                               )
        
#         Right now only good for fanduel
        self.current: pd.DataFrame = (pd
                                      .read_csv(f'../data/current-{mode}.csv', usecols=columns)
                                      .rename(columns, axis=1)
                                      .pipe(lambda df_: df_.loc[df_['name'].isin(injured_players)==False])
                                      .assign(name=lambda df_: df_.name.str.replace('.','',regex=False))
                                      .sort_values('name')
                                      .set_index('name')
                                     )
    
        if mode == 'draftkings':
            
            name_issues: dict[str,str] = {
                'KJ Martin': 'Kenyon Martin',
                'Guillermo Hernangomez': 'Willy Hernangomez',
            }

            self.current.index = self.current.index.map(lambda x: ' '.join(x.split(' ')[:2]))
            self.current.index = self.current.index.map(lambda x: name_issues.get(x,x))
            
            self.current: pd.DataFrame = (self.current
                                          .assign(
                                            pos=lambda df_: df_.pos
                                            .str.replace('/[GF]/UTIL','', regex=True)
                                            .str.replace('C/UTIL','C',regex=False)
                                            .str.replace('/[GF]', '', regex=True)
                                          )
                                         )
            
            def conv_dk_to_fd(pts, ast, trb, stl, blk, tov, three):
                
                result = sum([
                    1.0*pts,
                    1.5*ast,
                    1.25*trb,
                    2*stl,
                    2*blk,
                    -0.5*tov,
                    0.5*three,
                ])
                
                # result = 1.0*pts + 1.5*ast + 1.25*trb + 2*stl + 2*blk + -0.5*tov + 0.5*three
                
                if pts >= 10.0:
                    if ast >= 10.0 and trb >= 10.0:
                        result += 3
                    elif ast >=10.0 or trb >= 10.0:
                        result += 1.5
                        
                return result
            
            self.szn['fpts'] = self.szn[['pts', 'ast', 'trb', 'stl', 'blk', 'tov', '3p']].apply(lambda vals: conv_dk_to_fd(*vals), axis=1)
    
        self.players: tuple[str,...] = tuple(self.current.index)
    
    
    def add_projections(self):
        
        percs_df: pd.DataFrame = (self.szn
                                  .groupby('name')
                                  ['fpts']
                                  .agg([
                                      percentile(25), 
                                      percentile(50), # same as np.median
                                      percentile(75),
                                      'count',
                                      np.std
                                  ])
                                  .set_axis(['floor', 'med', 'ceiling', 'games', 'std'], axis=1)
                                  .pipe(lambda df_: df_.loc[df_.index.isin(self.players)])
                                 )
        
        # self.current: pd.DataFrame = (self.current
        #                               .loc[self.current.index.isin(percs_df.index)]
        #                              )
        
        return percs_df
        
        
    
    
    def load(self, **kwargs) -> pd.DataFrame:
        return (pd
                .concat([
                    self.current,
                    self.add_projections(),
                    # self.add_probabilities(),
                ], axis=1)
                .dropna()
                .sort_values('med', ascending=False)
                .assign(
                    games=lambda df_: df_.games.astype('uint16'),
                    f_per_dollar=lambda df_: 1000 * df_.floor / df_.salary,
                    med_per_dollar=lambda df_: 1000 * df_.med / df_.salary,
                    c_per_dollar=lambda df_: 1000 * df_.ceiling / df_.salary
                )
                .rename({
                    'med': 'median',
                    'f_per_dollar': 'floor/$',
                    'med_per_dollar': 'med/$',
                    'c_per_dollar': 'ceiling/$'
                }, axis=1)
                .round(2)
                .pipe(lambda df_: df_.loc[(df_['games'] >= 5) ])
                .sort_values(by=kwargs.get('sort', 'med/$'), ascending=False)
               )

In [5]:
outcomes = Outcomes(mode='fanduel')

In [6]:
outcomes.load(sort='floor/$')

,pos,salary,team,floor,median,ceiling,games,std,floor/$,med/$,ceiling/$
name,,,,,,,,,,,
Franz Wagner,SF/SG,5900,ORL,24.70,29.70,34.40,65,8.49,4.19,5.03,5.83
Kris Dunn,PG,5600,UTA,23.25,28.50,35.18,6,8.73,4.15,5.09,6.28
Domantas Sabonis,C/PF,10000,SAC,39.12,44.40,50.02,62,9.64,3.91,4.44,5.00
Julius Randle,PF,9800,NY,37.70,43.00,51.05,67,10.92,3.85,4.39,5.21
Jrue Holiday,PG,8300,MIL,31.70,37.00,47.00,53,11.53,3.82,4.46,5.66
Stephen Curry,PG,10500,GS,39.35,46.70,54.38,40,11.90,3.75,4.45,5.18
Kevon Looney,C,5100,GS,18.90,24.40,29.60,65,8.82,3.71,4.78,5.80
Jalen Green,SG/PG,6800,HOU,25.20,31.80,38.45,59,9.35,3.71,4.68,5.65
Damian Jones,C,3500,UTA,12.95,15.40,17.55,11,5.31,3.70,4.40,5.01
